In [2]:

# In the example below, we are going to optimize revenue produced by several classes of energy producing assets.
# If Windprice is the highest, followed by Solarprice and then Hydroprice, it implies that you want to prioritize 
# selling energy from the Wind asset due to its higher price, followed by Solar, and then finally Hydro.


In [1]:


from scipy.optimize import minimize
import numpy as np
import pandas as pd


pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Set a random seed for reproducibility
np.random.seed(42)

# Generate sample data for df_opt
num_records = 100

data = {
    'wind': np.random.randint(0, 100, size=num_records),
    'solar': np.random.randint(0, 100, size=num_records),
    'hydro': np.random.randint(0, 100, size=num_records),
    'Windprice': np.random.uniform(1, 10, size=num_records),
    'Solarprice': np.random.uniform(1, 10, size=num_records),
    'Hydroprice': np.random.uniform(1, 10, size=num_records)
}

df_opt = pd.DataFrame(data)

# Print the first few records of df_opt
print(df_opt.head())


# Define asset types and constraints
asset_types = ['Wind', 'Solar', 'Hydro']
lower_constraints = [0.05] * len(asset_types)
upper_constraints = [0.5] * len(asset_types)

# Define a function to optimize allocations for a given month
def optimize_allocations(month_data):
    # Extract prices for each asset type
    asset_prices = [month_data[f'{asset}price'] for asset in asset_types]

    # Sort asset types based on prices in descending order
    sorted_assets = [asset for _, asset in sorted(zip(asset_prices, asset_types), reverse=True)]

    # Define the objective function to minimize
    def objective_function(allocations):
        return -sum([allocations[i] * asset_prices[i] for i in range(len(asset_types))])


    # Define the equality constraint to ensure total allocation sums up to 1.0
    def equality_constraint(allocations):
        return sum(allocations) - 1.0

    # Create initial guesses for allocations
    initial_allocations = [1.0 / len(asset_types)] * len(asset_types)

    # Setup optimization problem with constraints
    constraints = ({'type': 'eq', 'fun': equality_constraint})
    bounds = [(lower, upper) for lower, upper in zip(lower_constraints, upper_constraints)]

    # Solve the optimization problem
    result = minimize(objective_function, initial_allocations, method='SLSQP', bounds=bounds, constraints=constraints)

    return result.x, sorted_assets  # Return both the optimized allocations and the sorted assets

# Apply the optimization function to each row in the DataFrame
optimized_allocations, sorted_assets = zip(*df_opt.apply(optimize_allocations, axis=1))

# Append optimized allocations to new columns in df_opt DataFrame
for i, asset_type in enumerate(asset_types):
    df_opt[f'Optimized_{asset_type}'] = [allocations[i] for allocations in optimized_allocations]

# Print sorted_assets for verification
print("Sorted Assets:")
print(sorted_assets)

print(type(df_opt))
df_opt.head(10)


   wind  solar  hydro  Windprice  Solarprice  Hydroprice
0    51     25      8   8.619950    3.443886    3.147372
1    92     88     61   8.706919    9.687266    7.118603
2    14     59     36   4.640573    5.115386    7.659179
3    71     40     96   8.989931    8.578208    3.144125
4    60     28     50   8.658356    2.749420    4.399560
Sorted Assets:
(['Wind', 'Solar', 'Hydro'], ['Solar', 'Wind', 'Hydro'], ['Hydro', 'Solar', 'Wind'], ['Wind', 'Solar', 'Hydro'], ['Wind', 'Hydro', 'Solar'], ['Wind', 'Hydro', 'Solar'], ['Wind', 'Solar', 'Hydro'], ['Wind', 'Hydro', 'Solar'], ['Wind', 'Hydro', 'Solar'], ['Solar', 'Wind', 'Hydro'], ['Wind', 'Solar', 'Hydro'], ['Wind', 'Hydro', 'Solar'], ['Hydro', 'Solar', 'Wind'], ['Solar', 'Hydro', 'Wind'], ['Hydro', 'Solar', 'Wind'], ['Hydro', 'Wind', 'Solar'], ['Wind', 'Hydro', 'Solar'], ['Hydro', 'Wind', 'Solar'], ['Solar', 'Hydro', 'Wind'], ['Solar', 'Wind', 'Hydro'], ['Hydro', 'Wind', 'Solar'], ['Solar', 'Hydro', 'Wind'], ['Wind', 'Solar', 'Hydro']

C:\Users\ryans\AppData\Roaming\Python\Python310\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


,wind,solar,hydro,Windprice,Solarprice,Hydroprice,Optimized_Wind,Optimized_Solar,Optimized_Hydro
0,51,25,8,8.619950,3.443886,3.147372,0.50,0.45,0.05
1,92,88,61,8.706919,9.687266,7.118603,0.45,0.50,0.05
2,14,59,36,4.640573,5.115386,7.659179,0.05,0.45,0.50
3,71,40,96,8.989931,8.578208,3.144125,0.50,0.45,0.05
4,60,28,50,8.658356,2.749420,4.399560,0.50,0.05,0.45
5,20,14,43,9.420715,4.702185,5.808947,0.50,0.05,0.45
6,82,44,23,8.068066,7.295610,5.469051,0.50,0.45,0.05
7,86,64,78,7.020894,2.245178,4.506563,0.50,0.05,0.45
8,74,88,58,6.226180,2.194709,3.678717,0.50,0.05,0.45
9,74,70,31,4.350545,9.725832,1.899864,0.45,0.50,0.05


In [3]:

# This prioritization aligns with maximizing revenue by selling energy from the asset with the highest price 
# first, and then the second highest next, and finally the third last. 
